## Installation

Use pip to install the Commonbase Python SDK

In [ ]:
pip install commonbase

## Usage


A Project ID and API Key are required for all Commonbase requests. You can find your project ID and generate an API key in the [Commonbase Dashboard](https://commonbase.com/).

### Functions

To use OpenAI's [Function Calling](https://platform.openai.com/docs/guides/gpt/function-calling) feature with Commonbase, make a call to `ChatCompletion.create` and provide a list of functions. It is very similar to using the OpenAI SDK.

In [3]:
import commonbase
import json

# In Production, you'd call a real weather API.
def get_current_weather(location: str, unit: str = "fahrenheit"):
    return {
        "location": location,
        "temperature": 72,
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }

messages: list[commonbase.ChatMessage] = [
    { "role": "user", "content": "What's the weather like in Boston?" }
]

response = commonbase.ChatCompletion.create(
    api_key="API_KEY",
    project_id="PROJECT_ID", 
    messages=messages,
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
)

best_choice = response.best_choice

# Check if GPT wanted to call a function
if (best_choice.function_call is not None):

    function_name = best_choice.function_call.name

    # The Commonbase SDK deserializes the arguments for you, though it may not always be valid JSON.
    # If the arguments are empty, check function_call.json["arguments"] to see the raw response from GPT.
    function_arguments = best_choice.function_call.arguments
    location, unit = function_arguments.get("location"), function_arguments.get("unit")

    # Only one function in this example, but you can have multiple.
    available_functions = {
        "get_current_weather": get_current_weather
    }   

    function_response = available_functions[function_name](location, unit)

    # Extend the conversation with the assistant's reply.
    messages.append(response.best_choice.to_assistant_chat_message())

    # Extend the conversation with the function response.
    messages.append({
        "role": "function",
        "name": function_name,
        "content": json.dumps(function_response)
    })

    # Get a new response from GPT where it can see the function response.
    second_response = commonbase.ChatCompletion.create(
        api_key="API_KEY",
        project_id="PROJECT_ID", 
        messages=messages
    )

    print(second_response.best_choice.text)

According to current weather data, the temperature in Boston, MA is 72°F and the forecast is sunny and windy.
